In [4]:
import os, random, math

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import lightgbm as lgb

import librosa
import librosa.display

from scipy.stats import skew, kurtosis
from sklearn.model_selection import train_test_split

#from prettytable import PrettyTable
from tqdm import tqdm_notebook, tqdm_pandas
tqdm_notebook().pandas(smoothing=0.7)

import IPython
import IPython.display as ipd

import matplotlib as mpl
mpl.rcParams['font.size'] = 14

from dataset import *
from util import Config, DataGenerator, audio_norm,getCacheTrainData, getCacheTestData

In [ ]:
ipd.Audio('../data/audio_train/00044347.wav')

In [ ]:
ipd.Audio('../data/audio_train_trimmed/00044347.wav')

In [13]:
train_root = '../data/audio_train/'
test_root = '../data/audio_test/'

train = pd.read_csv("../data/train.csv", index_col="fname")
test = pd.read_csv("../data/sample_submission.csv", index_col="fname")

LABELS = list(train.label.unique())
label_idx = {label: i for i, label in enumerate(LABELS)}
train["label_idx"] = train.label.apply(lambda x: label_idx[x])

mels = mfcc = 128
mixup = 2
mixup_alpha = 2
duration = 5
argucnt = 3
padfunc = 'constant'
config = Config(
        sampling_rate=16000, n_mels = mels, n_mfcc = mfcc,
        audio_duration=duration, n_folds=10, learning_rate=0.001,
        postfunc='', mixup=mixup, mixup_alpha = mixup_alpha,
        use_mfcc = True
    )

In [14]:
X_train_original, X_train_mel, X_train_mfcc, y =\
        get_cachedata_all_train(train, train_root, config, argucnt, padfunc)

../cache/wav_train_16000_5_9473_128_3.npy

wav dtype float16
mel dtype float16
mfcc dtype float16


In [20]:
X_test_original, X_test_mel, X_test_mfcc =\
        get_cachedata_all_test(test, test_root, config, padfunc)


wav dtype float16
mel dtype float16
mfcc dtype float16


In [22]:
del X_test_original

In [26]:
def combine_normalize(data1, data2):
    len_data1 = len(data1)
    data_all = np.vstack([data1, data2]).astype(np.float32)
    print(data_all.mean(), data_all.std())
    data_all = (data_all - data_all.mean()) / data_all.std()
    print(data_all.mean(), data_all.std())
    data1 = data_all[:len_data1]
    data2 = data_all[len_data1:]
    return data1, data2
    
d1, d2 = combine_normalize(X_train_mel, X_test_mel)

-59.88413 23.342537
1.0877094e-05 0.99997395


In [27]:
d1.shape

(37892, 128, 157, 1)

In [29]:
X_train_mel.shape

(37892, 128, 157, 1)